In [2]:
import networkx as nx
import numpy as np
import pandas as pd
from collections import defaultdict
import random
import json
import pickle as pkl

In [2]:
def load_entities(path="./"):
    
    id2aff_df     = pd.read_csv(path+"e_affiliation.csv")
    id2author_df  = pd.read_csv(path+"e_author.csv")
    id2concept_df = pd.read_csv(path+"e_concept.csv")
    auid2affid_df = pd.read_csv(path+"r_author2affiliation.csv")
    auid2conid_df = pd.read_csv(path+"r_author2concept.csv")
    
    id2aff     = {}
    id2author  = {}
    id2concept = {} 
    auid2affid = defaultdict(lambda x : -1)
    auid2conid = defaultdict(list)
    
    for index, row in id2aff_df.iterrows():
        id2aff[row["affiliationID:ID"]] = row["name"]
    for index, row in id2author_df.iterrows():
        id2author[row["authorID:ID"]]   = row["name"]
    for index, row in id2concept_df.iterrows():
        id2concept[row["conceptID:ID"]] = row["name"]
    for index, row in auid2affid_df.iterrows():
        auid2affid[row[":START_ID"]] = row[":END_ID"]
    for index, row in auid2conid_df.iterrows():
        auid2conid[row[":START_ID"]].append(row[":END_ID"])
    #     if row[":START_ID"] not in auid2conid:
    #         auid2conid[row[":START_ID"]] = [row[":END_ID"]]
    #     else:
    #         auid2conid[row[":START_ID"]].append(row[":END_ID"])
    id2aff[-1] = "None"
    
    return id2author, id2aff, id2concept, auid2affid, auid2conid

In [3]:
id2author, id2aff, id2concept, auid2affid, auid2conid = load_entities()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [42]:
import pickle as pkl

pkl.dump(id2author,  open("id2author.pkl",  "wb"))
pkl.dump(id2aff,     open("id2aff.pkl",     "wb"))
pkl.dump(id2concept, open("id2concept.pkl", "wb"))
pkl.dump(json.loads(json.dumps(auid2affid)), open("auid2affid.pkl", "wb"))
pkl.dump(auid2conid, open("auid2conid.pkl", "wb"))

In [3]:

id2author  = pkl.load(open("/Users/goojy/Documents/Project/GPT-GraphMaster/HIN/input/id2author.pkl", "rb"))
id2aff     = pkl.load(open("/Users/goojy/Documents/Project/GPT-GraphMaster/HIN/input/id2aff.pkl", "rb"))
id2concept = pkl.load(open("/Users/goojy/Documents/Project/GPT-GraphMaster/HIN/input/id2concept.pkl", "rb"))
auid2affid = pkl.load(open("/Users/goojy/Documents/Project/GPT-GraphMaster/HIN/input/auid2affid.pkl", "rb"))
auid2conid = pkl.load(open("/Users/goojy/Documents/Project/GPT-GraphMaster/HIN/input/auid2conid.pkl", "rb"))
r_coauthor = pd.read_csv("/Users/goojy/Documents/Project/GPT-GraphMaster/HIN/preprocessing/r_coauthor.csv")

In [4]:
author_from_id = r_coauthor[":START_ID"].values.tolist()
author_from    = [id2author[idx] for idx in author_from_id]
author_to_id   = r_coauthor[":END_ID"].values.tolist()
author_to      = [id2author[idx] for idx in author_to_id]

In [5]:
print(id2aff[3])
print(list(auid2affid.keys())[:100])
print(list(auid2conid.keys())[:100])

College of Geography Science, Nanjing Normal University, Nanjing 210097, China
['1', '2', '4', '5', '7', '8', '10', '13', '15', '20', '21', '22', '23', '27', '28', '29', '32', '33', '34', '35', '36', '37', '40', '41', '43', '44', '45', '46', '51', '52', '53', '66', '67', '68', '71', '74', '79', '80', '81', '90', '93', '95', '96', '97', '98', '99', '100', '101', '103', '105', '106', '108', '111', '113', '114', '115', '117', '129', '130', '131', '132', '138', '140', '141', '147', '148', '150', '151', '153', '156', '157', '158', '159', '163', '165', '169', '171', '173', '175', '178', '181', '189', '194', '195', '196', '199', '206', '207', '208', '214', '216', '218', '221', '222', '223', '225', '228', '231', '232', '240']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 

In [5]:
graph = nx.Graph()
nodes = [(id2author[key], {"affiliation": id2aff[auid2affid[str(key)]] if str(key) in auid2affid else "Nan", "research": ", ".join([id2concept[value] for value in auid2conid[key]][:3])}) for key in list(id2author.keys())]
graph.add_nodes_from(nodes)
edges = list(zip(author_from, author_to))
# edges = [(edge, {"relation": "collaboration"}) for edge in edges]
graph.add_edges_from(edges, relation="collaborate")

In [66]:
print(list(graph.nodes(data=True))[60:70])

[('Xinping Yan', {'affiliation': 'Nan', 'research': 'Intelligent Transport Systems, Intelligent Transportation Systems, Intelligent Transportation Systems Center'}), ('Tracey Rizzuto', {'affiliation': 'Nan', 'research': 'common error, Urban Legend, Common Errors'}), ('Jung Min Kong', {'affiliation': 'Nan', 'research': 'new data, Rule Refinement, Extended Data ExpressionThe rule'}), ('Raymond Leroy Springston', {'affiliation': 'Nan', 'research': 'CSE capstone design course'}), ('Lidija Petkovska', {'affiliation': 'Nan', 'research': 'dimensional field, electrical machine, permanent magnet'}), ('Hugo Ortega', {'affiliation': 'Columbia University', 'research': 'multimedia content, multimedia technology, ACM Multimedia Interactive Art'}), ('Chris Morrone', {'affiliation': 'Lawrence Livermore National Laboratory', 'research': 'Lustre file system, file system software, file system testing'}), ('Woochul Shin', {'affiliation': 'School of Computer Science, Inha University, Incheon, Korea', 'rese

In [8]:
from tqdm import tqdm
cnt = 0
for idx, node in tqdm(enumerate(list(graph.nodes(data=False))[:500])):
    nodesl = nx.bfs_successors(graph, node, depth_limit=4)
    nodesl = [nodel[0] for nodel in list(nodesl)][:20]
    if len(nodesl) < 10:
        continue
    subgraph = nx.subgraph(graph, nodesl)
    if subgraph.number_of_edges() > 50:
        continue
    nx.write_gml(subgraph, path="../input/GML/graph_"  + str(cnt) + ".gml")
    nx.write_graphml(subgraph, path="../input/GraphML/graph_"  + str(cnt) + ".graphml", named_key_ids=True, encoding='utf-8')
    nx.write_edgelist(subgraph, path="../input/EdgeList/graph_" + str(cnt) + ".edgelist", delimiter="\t", encoding="utf-8")
    nx.write_adjlist(subgraph, path="../input/AdjList/graph_"  + str(cnt) + ".adjlist", delimiter="\t", encoding="utf-8")
    cnt += 1

15it [00:02,  7.35it/s]